In [1]:
import pandas as pd
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
## Path of file
filename_test = "../data/clean/data.txt" ## file which want to give label [train data/test data]
filename_vocab = "../data/clean/skrip_kata_warung_final.xlsx"
filename_replace_word = "../data/clean/replace quantity.xlsx"

In [3]:
## Create list of vocabulary, list of vocabulary+label
df_vocab = pd.read_excel(filename_vocab)

list_vocab = []
list_vocab_label = []

for col in df_vocab.columns:
    for val in df_vocab[col].dropna():
        list_vocab.append(val.lower())
        list_vocab_label.append(val.lower()+':'+col)

In [4]:
## Load testing data
data = []
with open(filename_test, encoding='utf-8') as f:
    for i in f:
        data.append(i.strip())

In [5]:
## Replace 'Sesachet' -> 'satu sachet'
df_replace_word = pd.read_excel(filename_replace_word)
dict_replace_word = pd.Series(df_replace_word.after.values, index=df_replace_word.before).to_dict()

new_data = []

for line in tqdm(data):
    check_status = 0
    for key, val in dict_replace_word.items():
        if key in line:
            check_status = 1
            new_data.append(line.replace(key,val))
    if check_status != 1:
        new_data.append(line)

In [6]:
result = []
out_of_vocab = []

for i in tqdm(new_data):
    list_label = []
    tokenize_word = i.split()
    for token in tokenize_word :
        if token.lower() in list_vocab:
            if ((token.lower() == 'kopi') or (token.lower() == 'susu')) and (tokenize_word.index(token) == 0):
                list_label.append('ITEM')
            elif (token.lower() == 'delapan') and ('Tujuh' not in i):
                list_label.append('NUMERIC')
            elif (token.lower() == 'cap') and ('Lang' not in i):
                list_label.append('UOM')
            elif (token.lower() == 'kopi') or (token.lower() == 'susu'):
                list_label.append('DESCRIPTION')
            elif ('Dua Belas' in i) and ((token.lower() == 'dua') or (token.lower() == 'belas')):
                list_label.append('DESCRIPTION')
            elif ('Enam Belas' in i) and ((token.lower() == 'enam') or (token.lower() == 'belas')):
                list_label.append('DESCRIPTION')
            elif ('Dua Puluh' in i) and ((token.lower() == 'dua') or (token.lower() == 'puluh')):
                list_label.append('DESCRIPTION')
            elif (('Tujuh Enam' in i)or('Tujuh Puluh Enam' in i)) and ((token.lower()=='tujuh')or(token.lower()=='puluh')or(token.lower()=='enam')):
                list_label.append('DESCRIPTION')
            elif (token.lower() == 'bos') and ('Nutri' not in i):
                list_label.append('SOP-TRIGGER')
            elif (token.lower() == 'dua') and ('MX' not in i):
                list_label.append('NUMERIC')
            elif (token.lower() == 'tiga') and ('Kaki' not in i):
                list_label.append('NUMERIC')
            elif (token.lower() == 'wan') and ('Yusi' not in i):
                list_label.append('NUMERIC')
            elif (token.lower() == 'tujuh') and ('Delapan' not in i):
                list_label.append('NUMERIC')
            elif ((token.lower() == 'ekstra') or (token.lower() == 'ektra')) and (tokenize_word.index(token) == 1):
                list_label.append('DESCRIPTION')
            else :
                idx_vocab = list_vocab.index(token.lower())
                list_label.append(list_vocab_label[idx_vocab].split(':')[1])
        else :
            out_of_vocab.append(token)
    result.append(i+':'+(' '.join(list_label)))

out_of_vocab = set(out_of_vocab)

In [7]:
out_of_vocab

set()

In [25]:
with open('../data/clean/data-with-label.txt','w') as outfile:
    for line in tqdm(result):
        outfile.write('%s\n' % line)

In [26]:
result[:5]

['Abese Kopi Susu Sachet Satu Saset:BRAND DESCRIPTION DESCRIPTION UOM NUMERIC UOM',
 'Abese Kopi Susu Sachet Atu Saset:BRAND DESCRIPTION DESCRIPTION UOM NUMERIC UOM',
 'Abese Kopi Susu Sachet Tu Saset:BRAND DESCRIPTION DESCRIPTION UOM NUMERIC UOM',
 'Abese Kopi Susu Sachet Dua Saset:BRAND DESCRIPTION DESCRIPTION UOM NUMERIC UOM',
 'Abese Kopi Susu Sachet Dobel Saset:BRAND DESCRIPTION DESCRIPTION UOM NUMERIC UOM']

In [17]:
len(result)

261375